### Preface:

The separation of signal from noise is fundamental to the study of machine learning, signal processing, and empirical analysis of dynamical systems. The following is an analysis on the effects of noise in discriminatory tasks under a convolutional nueral network (cnn) model. There are three parts. The first trains and analyzes the model itself in the absence of noise. The second introduces (additive Gaussian) noise into the training set during training, and observes the affect on the accuracy of the model upon testing. The last part investigates the affect of label noise during training and its relationship to model accuracy. Each part consists of a computational solution (See Analysis notebook), selected suggestions and approaches in a real world context, followed by extensions of the analysis with references where explicit calculation or elaboration is absent.

### Part 1: Classifier on Original Data

**Directions:** Use the predefined split of training and test sets, train a convolutional neural network classifier on the MNIST training set and test on the test set.

* What is your test set error rate?
The test error rate is:  0.028
* What is the test set error rate for each class?
The test error rates for classes 0 - 9: [ 0.005  0.005  0.037  0.036  0.021  0.015  0.023  0.027  0.037  0.073]

*Are some classes more challenging than others to distinguish from each other? Why?

Classes 10, 9, 3, and 4 seem to be the most difficult to distinguish. It seems reasonable that classes 9 and 3 are at least visually easily confused with other classes. As for "which classes" tend to be misclassified as "which," plotting true label vs. class assignment in a confusion matrix helps visualize an answer to this. Unfortunately, the fast convergence of this particular model makes it very difficult to see this. The notebook will need to be run on train, test sizes that are smaller (1000, 500 worked better) in order to illustrate this. Moreover, since random initialization makes training nondeterministic, you might need to do this several times and average the results to get an idea of what is training noise and what is actually do slower discrimination of classes on average.

Though there is still an open debate (my current impression) regarding whether or not a general dataset can be imbedded as a manifold in some feature space, it does make sense to think about how the cnn encodes each image in the final layer before discrimination. The L2 distance between these 'nueral codes' and original L2 image distances between raw images can then be compared qualitatively with embedding methods such as (t-SNE), which imbed the training set in two dimensions. Finally, you can roughly compare this notion of distance with the original distance by adding a heat map to the borders of adjacent images in the plane which correspond to the original raw L2 image distance between neighbors. All such embedding methods (to my knowledge) are not unique (t-SNE optimizes a nonconvex objective), and more work research will be needed to come up with a more rigorous, consistent, and insightful description of the geometric properties of cnns.

Related Extensions: Topological persistence applied in CNN.
See: https://arxiv.org/pdf/1608.07373.pdf

* Based only on information gathered in the first epoch of training, do you think that the model would benefit from more training time? Why?

This model 'converged' very quickly. A properly regularized model should continue to converge if you train longer. The questions are whether you need better performance, whether its easy to get, and whether it looks plausible or comes at a reasonable cost. The cnn only trained for one epoch and already loss function has begun to plataue. In general, this is not a sure sign that the network is converging (See: Karpathy's https://lossfunctions.tumblr.com/); however, this simple model already achieves over 97% accurracy. Training more will not cost much more resources, time or money, so training more seems like a good idea. Generally speaking, you should be using a validation set, and monitoring the loss function as well as the validation accuracy compared to to the training accuracy. This difference will indicate whether your model is poorly regularized and is over fitting. Early stopping allows one to automatically stop training when the validation accuracy stops improving long enough to exhaust a patience parameter, which is depleted over long periods without improvement.

See the cs231n git page for some illustrative graphs: http://cs231n.github.io/neural-networks-3/

* Besides training for a longer time, what would you do to improve accuracy?

There are finite (but numerous) factors and practices which not only help one choose an architecture and other design parameters but contribute to the optimal tuning of cnn. Many of these are empirical rules. As a start, the cs231n, Neural Networks parts 2 and 3 pages provide an excellent elaboration on these topics: http://cs231n.github.io/. One is also encourages to take a look at Yann Lecun's page (http://yann.lecun.com/exdb/mnist/), which provides a history of improvements on the MNIST benchmark as well as links to the appropriate publicaitons.

Otherwise, here are is an an outline of a number of components which help improve results in an end to end deep learning system.

1. Data Collection:
    * More or better quality data
2. Preprocessing and Augmentation:
    * Data Augmentation: Rotations, Translations, Deformations, Noise
3. Training:
    * Parameter Updates:         
        * Optimizers: SGD + Nesterov Momentum or Adam
        * Learning Rate Annealing
    * Weight Initialization: 
        * Unsupervised pretraining 
        * Xavier Initialization 
    * Hyperparameter Optimization:  
        * Random search
    * Regularization and Cost Function Choices:
    * Other Delta Improvements
        * Architecture: Add Drop Out, Batch Normalization
4. Extra Performance:
    * Form Model Ensembles
    
Note: The model used in this project is creditted to François Chollet. No tweaking to the hyperparameters was done to avoind overfitting the test set. The model can be found here: https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

### Solution Part1: Classifier on Original Data

* Save model weights at the end of a single epoch.
    * Use this to save the class assignments for each test set point
    * Use class assignments to plot confusion matrix
    * Compare misclassification rates as a bar chart for each classe
* After each batch plot the test error. Demonstrate a downward sloping trend towards end.
* Perform crossvalidation and plot the crossvalidation and test error for each batch. Demonstrate high capacity characteristic/high variance ... decrease with more training. Crossvalidation, parameter tuning, possibly data set augmentation although not really for mnist??.

### Part 2: Added Image Noise

**Directions:** Now add random Gaussian noise to the training set images. Set the mean to 0 and the standard deviation to 8 (given an input image range between 0 and 255). Ensure that the output image pixel image values are properly clipped between 0 and 255. Retrain the model on the noise-added images and test it on the original (noise-free) test set. Now what is your error rate?  

Repeat the process for two more standard deviation values: 32 and 128. Plot the error rate as a function of noise standard deviation.  

* What are the implications of the dependence of accuracy on noise if you were to deploy a production classifier? How much noise do you think a production classifier could tolerate?

At one level, the increasing trend in the error vs. noise plot implies a worsening in discrimination with image corruption. If there are enough data points to infer a trend, a concave trend would imply that the cnn is somewhat robust against image noise, while a convex trend would imply less error rate stability with respect to increases in noise. Finally, a linear trend would imply that the error rate is affected by changes in noise in the same proportion regardless of the baseline image corruption.

In a production setting, there will ultimately be specified tolerances for the error rate and robustness of a classifier. With this in mind, it can be important to view the end to end deep learning pipeline from a systems engineering perspective. It then becomes clear that both bounding the error rate (and its corresponding uncertainty) as well as the higher derivatives of the error rate (and their uncertainties) become exceedingly import for other design specifications, such as robustness. Depending on the nature of the application, these factors, when not taken into account, may act as multipliers to other sources systematic and random error within the pipeline, potentially compromising design specifications.

The noise tolerance of production classifier is application specific. In the MNIST toy problem, a sigma of 8 seems to be about the limit if we want to keep the error rate around 45%. For high risk applications such as choices over large crops in agriculture or medical diagnosis, the tolerance could be much lower or at least biased against false negatives. When thinking about how to set the tolerance for the application, the sensitivity of the system to the classification result is important as well as the immediate affects of the classifier and any other external constraints, such as legal regulations.

* Do you think that Gaussian noise is an appropriate model for real-world noise if the characters were acquired by standard digital photography? If so, in what situations? How would you compensate for it?

In digital image acquisition, sources of Gaussian noise seem to most often arise from sensors noise caused by poor illumination and/or high temperature and/or from electronic transmission noise within the camera's circuits. Traditinally, noise compensation is performed in a preprocessing step via filtering. When the noise is additive, spatial filtering tends to be a method of choice.[1]

I will add that attention to and control of environmental variables during data acquisition is also a practical preventative measure.

There are a whole suit of spatial filters which are more optimal for different types of noise. In the analysis notebook, two local filters (mean and Gaussian types) are demonstrated along with a weiner filter, which happens to filter most optimally for additive gaussian noise. The results are compared visually and the error rates for a trained network are compared for the weiner filter and the unfiltered data.

In addition to basic filtering techniques, data augmentation can help a classifier become more robust to particular types of noise. So it may be reasonable, to test and compare classifiers trained on data sets augmented with noisy data. Even without having plotted affect on the loss functions, this is a senario that would merit more training epochs. There are also machine learning approaches which provide various approaches for extracting signal from noise. For additinoal references on separating signal from noise in image processing, see references below.


* Is the accuracy of certain classes affected more by image noise than others? Why?

Going back to the previous discussion, it's important to understand what a cnn 'sees' as 'similarity' between different images, in some sense how strongly the preimages of different classes overlap for different network scores under a given model. To this end there are a few entertaining papers that people like to refer to that describe how neural networks can be easily fooled by unrecognizable images. In the one cited here the researches, use essentially 'map out' a partial preimage of the network to discover unrecognizable images which lead to high confidence predicitions on known image classes (https://arxiv.org/pdf/1412.1897.pdf). Generally speaking, it seems reasonable that classes which are easily confused (or are closer to each other by some measure, such as t-SNE embedding, etc.) may contain images whose may be more sensitive to noise.

### Part 3: Label Noise

**Directions:** Go back to the original, noise-free MNIST data set. Now take 5% of the training images and randomize their labels. Retrain your classifier on the label-randomized images and test on the original (non-randomized) test set.  

Then repeat the process with random labels for 15% and 50% of the training images. Plot the error rate as a function of randomized label fraction.  

* How important are accurate training labels to classifier accuracy?
Again this is application specific, but, for the sake of argument, let's choose the error tolerance to be 33% (Note that we should not reuse the tolerance from part 2 due to a lack of data). Then going off of that single data point, we can tolerate a label error rate of no more than 15%.

* How would you compensate for label noise? Assume you have a large budget available but you want to use it as efficiently as possible.

First, I would construct an experiment to determine the amount of noise in the data labels. I would pay attention to whether there are any systematic features, such as are present in many real world data sets which affect mislabeling. After constructing a plausible model of the label noise distrubtion as a function of these features, I might try one of at least three methods generally starting with whichever is most simple and cost effective. The first method, which I describe in more detail in the next question, involves density estimation of this discrete distribution by systematically introucing mislabled data into a cleaned subset of the training data. The second approach would work best in the case of uniform random noise accross class labels. Simply put, you just design an experiment to approximate the proportion $\hat{p}$ of mislabeled data points across all training set. If you know the size of your training set to be $N$, and you desire a specified error $\epsilon$ on the estimator $\hat{p}$ with a corresponding $(1-\alpha)\%$ confidence interval, then we will need a sample size that is at least $n=\dfrac{m}{1+\dfrac{m-1}{N}}$, where $m=\dfrac{z^2_{\alpha/2}\hat{p}(1-\hat{p})}{\epsilon^2}$ is the upper bound on the sample size in the limit that $N >> m$. Finally, the last approach is to adjust your the classification pipeline to incorporated models which are intrinsically more robust to noise. In "Training Deep Neural Networks with Noisy Labels with Bootstrapping," the discussion includes methods for reweighting the loss function to incorporate a class assignment. In "Learning from Massive Noisy Labeled Data for Image Classification" describes an end to end solution which learns label mislabeling and adjusts for these errors with backpropagation.


* How would you quantify the amount of label noise if you had a noisy data set?
The percent of mislabeled data across various classes can be bounded for a specified confidence by designing an experiment as described above. Another approach would be to try training another (perhaps an ensemble of) classifiers on a clean data set (not necessarily the primary one) and then to evaluate a small cleaned fraction of your data set on these 'auxillary classifiers.' As you continue to systematically introduce mislabeled images into these test sets, you may be able to fit the observed classification error rate with respect to percent label error for this diagnostic model with high enough confidence so as to use this result as a diagnostic on the full data set. Another approach would be to attach the problem via density estimation in order to learn the full discrete probability distribution across the labels.

* If your real-world data had both image noise and label noise, which would you be more concerned about? Which is easier to compensate for?

I still am biased toward image noise over label noise, even though the results of this example are currently conflicting with this sentiment. It is my understanding that this type of noise is generally speaking more difficult to remove or adjust the data or machine learning architecture to compensate for. At least one of the references that I provide below mention label noise as being more critical and difficult to deal with than image noise. As far as this experiment is concerned, the model at least needs to be trained longer and over more data points in the image and label noise domain in order to come to a more conclusive result.

### Part 4: Conclusions and Further Work:

It is well known that dealing with the detection, removal, and implications of different types of noise in an engineering and production context is a endevour which spans many areas of analytic study. Statistics, mathematical optimization, digital signal processing, machine learning, systems engineering, and other bodies of thought all come to bear on this subject that has been the underpinnings of data driven scientific achievement throughout history and is fundamental to the notion of learning. The approaches to dealing with these problems are vast and diverse. Nevertheless, within the enormous body of literature there is a finite set of tools, some of which are described here, which can be greedily sorted through and prioritized under the objectives of real world production constraints and a fiscal bottom line. In this discussion, I have presented to first order a number of solutions that I think could be most practical in context of the stated questions resource constraints of this analysis.  

Going forward, the first steps would of course be to optimize and expand the computations already presented here. Crossvalidation, hyperparameter tuning, and model architecture are all factors that play into a more thorough approach. Furthermore, more data points are needed to fit trends to the error rate vs noise data. Some of this proceedure along with additional insights is outlined in the Stanford CS231n class report below. After this, there are many directions both theoretically and practically which would help better specify the problem and illuminate a path forward towards more predictive computation. 

Thank you!

Note/Work Undone: This notebook still needs to be error checked. In addition to the discussion section, the Analysis notebook and pipeline package do not adhere to the Python style guide. Furthermore, several pieces of the code within the analysis notebook are copied verbatim repeatedly instead of encapsulted in a function.

# Selected Literature on Image Noise and Robustness:

Deep Neural Networks are Easily Fooled: High Confidence Predictions for Unrecognizable Images  
https://arxiv.org/pdf/1412.1897.pdf

Understanding How Image Quality Affects Deep Neural Networks:  
https://arxiv.org/pdf/1604.04004.pdf

Robustness of Deep Convolutional Neural Networks for Image:  
Recognition http://www.springer.com/cda/content/document/cda_downloaddocument/9783319304465-c2.pdf?SGWID=0-0-45-1551588-p179871679

Image De-raining Using a Conditional Generative Adversarial Network  
He Zhang, Student Member, IEEE, Vishwanath Sindagi, Student Member, IEEE  
Vishal M. Patel, Senior Member, IEEE  
https://arxiv.org/pdf/1701.05957.pdf

# Selected Relevant Literature on Label Noise:

A Stanford CS231n Report - On the Robustness of ConvNets to Training on Noisy Labels: David Flatow, Daniel Penner
http://cs231n.stanford.edu/reports/flatow_penner_report.pdf


Training Convolutional Networks with Noisy Labels
Sainbayar Sukhbaatar, Joan Bruna, Manohar Paluri, Lubomir Bourdev, Rob Fergus
https://arxiv.org/abs/1406.2080

Learning with Noisy Labels
https://papers.nips.cc/paper/5073-learning-with-noisy-labels.pdf

TRAINING DEEP NEURAL NETWORKS
ON NOISY LABELS WITH BOOTSTRAPPING
Scott E. Reed & Honglak Lee, Et Al.
https://arxiv.org/pdf/1412.6596.pdf


Auxilliary Image Regularization for Deep CNNs with Noisy labels:
Samaneh Azadi1, Jiashi Feng1, Stefanie Jegelka & Trevor Darrell1
https://arxiv.org/pdf/1511.07069.pdf

Learning from Massive Noisy Labeled Data for Image Classification:
Tong Xiao, Tian Xia, Yi Yang, Chang Huang, and Xiaogang Wang
The Chinese University of Hong Kong, Baidu Research
http://www.ics.uci.edu/~yyang8/research/noisy-label/noisy-label-cvpr2015.pdf

### References:

Image Restoration Talk: Prof. Dr. Philippe Cattin, University of Basel  
[1] https://miac.unibas.ch/SIP/06-Restoration.html#(14)

CS231n: Highly recommended reference on CNNs and DL principles:  
Course Notes:  
http://cs231n.github.io/  
Lecture Videos:  
https://www.youtube.com/playlist?list=PLIUoqCcJd2BjsI11qafvMWv_UqiH1Wu3Q  

### Code Citations:

t-SNE Embedding:
http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

François Chollet, Creater of Keras:
https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

CNN Weights and Activation Visualization Example:
https://github.com/julienr/ipynb_playground/blob/master/keras/convmnist/keras_cnn_mnist_v1.ipynb

The Keras docs:
https://keras.io/

The scikit learn User Guide:
http://scikit-learn.org/stable/user_guide.html

Scipy Docs:
https://docs.scipy.org/doc/numpy/reference/

Matplotlib Docs:
http://matplotlib.org/contents.html
